* TODO : AI-HUB `차량 및 사람 인지 영상`에서 다운로드 [링크](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=195)
* Lane Detection for Carla Driving Simulator 데이터셋 다운로드 [링크](https://www.kaggle.com/datasets/thomasfermi/lane-detection-for-carla-driving-simulator)

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
cd C:/Users/abc/

In [ ]:
import os
import shutil
import json
import random
import yaml
import os
import cv2
import torch
import numpy as np
import os


In [5]:
# 서울특별시, 구, 날씨, 시간대, 장소를 각각 리스트로 정의하는 코드

city = ['서울특별시']
gu = ['영등포구', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구','은평구', \
  '서대문구', '마포구', '양천구', '강서구', '구로구','금천구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']
weather = ['맑음']
gan = ['주간', '야간']
sil = ['실내', '실외']

In [13]:
# 'label' 폴더의 파일 이름을 변환하여 'labels' 폴더로 복사하는 코드

root_folder = 'label'
change_folder = 'labels'

if not os.path.exists(change_folder):
      os.makedirs(change_folder)

file_list = os.listdir(root_folder)
for file in file_list:
  name, ext = file.split('.')
  p = name.split("_")
  p[1] = str(city.index(p[1]))
  p[3] = f"{gu.index(p[3]):02d}"
  p[4] = str(weather.index(p[4]))
  p[5] = str(gan.index(p[5]))
  p[6] = str(sil.index(p[6]))
  new_name = "_".join(p) + '.' + ext
  shutil.copyfile(os.path.join(root_folder, file), os.path.join(change_folder, new_name))


In [7]:
# 'img' 폴더의 이미지 파일 이름을 변환하여 'images' 폴더로 복사하는 코드

root_folder = 'img'
change_folder = 'images'

if not os.path.exists(change_folder):
      os.makedirs(change_folder)

file_list = os.listdir(root_folder)
for file in file_list:
  name, ext = file.split('.')
  p = name.split("_")
  p[1] = str(city.index(p[1]))
  p[3] = f"{gu.index(p[3]):02d}"
  p[4] = str(weather.index(p[4]))
  p[5] = str(gan.index(p[5]))
  p[6] = str(sil.index(p[6]))
  new_name = "_".join(p) + '.' + ext
  shutil.copyfile(os.path.join(root_folder, file), os.path.join(change_folder, new_name))

In [14]:
# XML 형식의 바운딩 박스 좌표를 YOLO 형식으로 변환하는 코드

def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = float(((bbox[2] + bbox[0]) / 2) / w)
    y_center = float(((bbox[3] + bbox[1]) / 2) / h)
    width = float((bbox[2] - bbox[0]) / w)
    height = float((bbox[3] - bbox[1]) / h)
    return [x_center, y_center, width, height]

In [16]:
# JSON 형식의 라벨 파일을 YOLO 형식으로 변환하여 새 텍스트 파일로 저장하는 코드

label_folder = './labels/'
new_folder = './new_labels/'

# 새로운 폴더가 없으면 생성
if not os.path.isdir(new_folder):
    os.mkdir(new_folder)

# 클래스와 차량 종류 정의
classes = ['car', 'pedestrian']
cars = ['일반차량', '목적차량(특장차)', '이륜차']
label_list = os.listdir(label_folder)

# 라벨 파일 읽고 변환
for file in label_list:
    try:
        with open(os.path.join(label_folder, file), 'r', encoding='utf-8') as json_file:
            lab = json.load(json_file)
            h = int(lab['camera']['resolution_height'])
            w = int(lab['camera']['resolution_width'])
            f = open(os.path.join(new_folder, file[:-4]+'txt'), 'a')
            for l in lab['annotations']:
                if l['label'] in cars or l['label'] == '보행자':
                    if l['label'] in cars: 
                        idx = 0
                    else:
                        idx = 1
                    points = l['points']
                    yolo_bbox = xml_to_yolo_bbox([int(points[0][0]), int(points[0][1]), int(points[2][0]), int(points[2][1])], w, h)
                    bbox_string = " ".join([str(x) for x in yolo_bbox])
                    result = f"{idx} {bbox_string}\n"
                    f.write(result)
            f.close()
    except Exception as e:
        print(f"파일 {file}을(를) 읽는 중 오류 발생: {e}")


In [17]:
# 기존의 'train' 및 'val' 폴더를 삭제하고, 새로운 폴더 구조를 생성하는 코드

if os.path.isdir('./train'):
    shutil.rmtree('./train')
if os.path.isdir('./val'):    
    shutil.rmtree('./val')

folder_list = ['train', 'val', 'train/images', 'train/labels', 'val/images', 'val/labels']
for folder in folder_list:
    if not os.path.isdir(folder):
        os.mkdir(folder)

In [19]:
# 데이터셋을 랜덤하게 분할하여 10%를 검증용(val)으로, 나머지를 학습용(train)으로 복사하는 코드

random.seed(2020)

root_folder = 'C:/Users/abc/'
file_folder = 'new_labels'

file_list = os.listdir(file_folder)
random.shuffle(file_list)
test_ratio = 0.1

val_list = file_list[:int(len(file_list)*test_ratio)]
train_list = file_list[int(len(file_list)*test_ratio):]

for i in val_list:
    from_file_path = os.path.join(root_folder, 'images', i[:-4]+'.png')
    to_file_path =  os.path.join(root_folder, 'val/images/',i[:-4]+'.png')
    shutil.copyfile(from_file_path, to_file_path)
    from_file_path = os.path.join(root_folder, file_folder, i)
    to_file_path =  os.path.join(root_folder, 'val/labels/',i)
    shutil.copyfile(from_file_path, to_file_path)

for i in train_list:
    from_file_path = os.path.join(root_folder, 'images', i[:-4]+'.png')
    to_file_path =  os.path.join(root_folder, 'train/images/',i[:-4]+'.png')
    shutil.copyfile(from_file_path, to_file_path)
    from_file_path = os.path.join(root_folder, file_folder, i)
    to_file_path =  os.path.join(root_folder, 'train/labels/',i)
    shutil.copyfile(from_file_path, to_file_path)

print(len(os.listdir('train/images')))
print(len(os.listdir('train/labels')))
print(len(os.listdir('val/images')))
print(len(os.listdir('val/labels')))

In [22]:
# 데이터셋 경로와 클래스 정보를 YAML 형식으로 저장하는 코드

data = dict()

data['train'] = 'C:/Users/abc/train'
data['val'] = 'C:/Users/abc/val'
data['test'] = 'C:/Users/abc/val'

data['nc'] = 2
data['names'] =['car','pedestrian']


with open('car_detection.yaml', 'w') as f:
	yaml.dump(data, f)

In [ ]:
# 라벨 이미지를 읽어 두 개의 마스크 이미지로 변환하여 저장하는 코드
# 왼쪽 차선(1)과 오른쪽 차선(2)에 대한 마스크를 각각 생성

dir_path = 'C:/Users/abc/'
save_path = 'C:/Users/abc/'
label_images = os.listdir(dir_path)

if not os.path.exists(os.path.join(save_path, 'masks/0')):
      os.makedirs(os.path.join(save_path, 'masks/0'))

if not os.path.exists(os.path.join(save_path, 'masks/1')):
      os.makedirs(os.path.join(save_path, 'masks/1'))
      
for file in label_images:
    img = cv2.imread(os.path.join(dir_path, file))
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    mask_l = np.zeros_like(gray_img)
    mask_r = np.zeros_like(gray_img)
    for i in range(gray_img.shape[0]):
        for j in range(gray_img.shape[1]):
            if gray_img[i][j] == 1:
                mask_l[i][j]= 255
            else:
                mask_l[i][j] = 0
            if gray_img[i][j] == 2:
                mask_r[i][j]= 255
            else:
                mask_r[i][j] = 0
    cv2.imwrite(os.path.join(save_path, 'masks/0', file), mask_l)
    cv2.imwrite(os.path.join(save_path, 'masks/1', file), mask_r)

In [ ]:
# 라벨 이미지 파일 이름에서 '_label'을 제거하여 이름을 변경하는 코드
# 왼쪽 차선(0)과 오른쪽 차선(1) 마스크 이미지의 이름을 동시에 수정

dir_path1 = 'C:/Users/abc/masks/1'
dir_path0 = 'C:/Users/abc/masks/0'

label_images = os.listdir(dir_path)
for file in label_images:
    new = file.replace('_label', '')
    os.rename(os.path.join(dir_path0, file), os.path.join(dir_path0, new))
    os.rename(os.path.join(dir_path1, file), os.path.join(dir_path1, new))

In [ ]:
cd C:/Users/abc/pytorch-nested-unet

In [36]:
# 이미지 및 마스크 디렉토리를 설정하는 코드
# 이미지 디렉토리와 왼쪽(0) 및 오른쪽(1) 차선 마스크 디렉토리의 경로를 지정

import matplotlib.pyplot as plt
%matplotlib inline

DATA_DIR = './inputs/lane/'

image_dir = os.path.join(DATA_DIR, 'images')
mask_dir_l = os.path.join(DATA_DIR, 'masks/0')
mask_dir_r = os.path.join(DATA_DIR, 'masks/1')

In [ ]:
cd C:/Users/abc/

In [ ]:
# PyTorch와 CUDA 관련 정보를 출력하는 코드

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Number of CUDA devices: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"CUDA Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No CUDA devices detected.")

# 추가적인 CUDA 정보 출력
if torch.cuda.is_available():
    current_device = torch.cuda.current_device()
    print(f"Current CUDA device: {current_device}")
    print(f"Device capability: {torch.cuda.get_device_capability(current_device)}")
    print(f"Total memory: {torch.cuda.get_device_properties(current_device).total_memory / 1e9:.2f} GB")

In [ ]:
# YOLOv8 모델을 설정하고 주어진 데이터셋으로 훈련하는 코드
# 훈련 데이터와 하이퍼파라미터를 설정하고, CUDA를 사용하여 훈련을 수행

from ultralytics import YOLO
model = YOLO('yolov8s.yaml')
results = model.train(
    data='C:/Users/abc/car_detection.yaml',
    epochs=4,
    batch=16,
    device='cuda',
    patience=30,
    name='car_detection_abc',
    # amp=False  # AMP 활성화
)

In [ ]:
cd C:/Users/abc/

In [ ]:
# 저장된 YOLO 모델을 불러와 주어진 데이터셋으로 검증하는 코드
# 검증 결과로 mAP@50-95 및 mAP@50 값을 출력

from ultralytics import YOLO
import torch

# 모델 경로
best_model_path = 'cd C:/Users/abc/car_detection_abc.pt'
# 데이터셋 구성 파일 경로 (yaml 파일 경로)
data_path = 'C:/Users/abc/car_detection.yaml'

# 모델 불러오기
model = YOLO(best_model_path) 

# 모델 검증 수행
try:
    results = model.val(data=data_path, verbose=True)
    
    # 결과 출력
    print("\nValidation Results:")
    print(f"mAP@50-95: {results.box.map:.4f}")
    print(f"mAP@50: {results.box.map50:.4f}")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please check your model file and data path.")


In [ ]:
cd C:/Users/abc/pytorch-nested-unet

In [ ]:
# NestedUNet 아키텍처를 사용하여 주어진 데이터셋으로 모델을 훈련하는 명령
# 훈련에 필요한 하이퍼파라미터를 설정 (에포크, 배치 크기, 입력 크기, 옵티마이저 등)

!python train.py --dataset lane --arch NestedUNet --name lane_segmentation --epochs 123 --batch_size 123 --input_w 123 --input_h 123 --optimizer Adam --num_classes 123 --lr 123